In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import datetime 
import time 
import re 
import csv                        # Convert file to csv and save to local directory 
import json                       # JSON manipualtion
import requests                   # Handles Pushshift HTTP requests 
import os 
import datetime 

import pandas as pd
import requests            
import json                    
import csv                        
import re
import time
import datetime

In [4]:
# Install base-36 
!pip install base36 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
# Install PRAW and CONVOKIT 
!pip install praw 
!pip install convokit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 188 kB 5.1 MB/s 
     |████████████████████████████████| 54 kB 2.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 167 kB 4.8 MB/s 
     |████████████████████████████████| 235 kB 37.3 MB/s 
     |████████████████████████████████| 175 kB 43.0 MB/s 
     |████████████████████████████████| 53 kB 1.8 MB/s 
  Created wheel for convokit: filename=convokit-2.5.3-py3-none-any.whl size=204129 sha256=f08a4ffb7cde55326f5b9f4c51422df38fb966b492dc6de339d6bfc7c3ea41e9
  Stored in directory: /root/.cache/pip/wheels/b0/e8/2d/81c4477fe586fe4dad2de2886b990e90e839ffccd5158ed0f3
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=b980a0159bfc9f9a01cac2230021f8093f8d64a8dcc75c2c2baa5275204d1e88
  Stored in directory: /root/.cache/pip/wheels/8a

In [6]:
!pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
from datetime import datetime
import matplotlib.pyplot as plt  
import praw
from tqdm import tqdm
import base36
from convokit import Corpus, download

## REFERENCES
* The following reference has served as a guide for this task
https://github.com/CornellNLP/ConvoKit/blob/master/examples/corpus_from_pandas.ipynb

In [8]:
reddit = praw.Reddit(                                                           # Initializing the praw instance 
                     client_id = "ZpfGWwr1U_CoS7Qyc6R4YA",                          
                     client_secret = "gw-I0KmsINO14qjeuhnmCSp9kQtkAQ", 
                     username = "sinhasagar507", 
                     passowrd = "Maa@g110", 
                     user_agent = "personal use script",
                     check_for_async = False
)

In [9]:
# Building the reddit comment extraction using PSAW and performing extraction for specific IDs
def get_PSAW_Comment_Data(comment_id) -> dict():
    """This function fetches comment metadata for each submission"""
    
    comment_endpoint = "https://api.pushshift.io/reddit/comment/search?ids="+str(comment_id)
    print(comment_endpoint)
    r = requests.get(comment_endpoint)
    data = json.loads(r.text)
    return data["data"]

In [24]:
def saveCommentIds(root_path): 
  """This function saves list of comment IDs for each submission ID to file"""
  print("Enter file name to save the submission Id and comment Ids") 
  print("Don't forget to add the .txt extension. I repeat, add the .txt extension")    
  filename = input()
  file_path = str(root_path) + filename

  with open(file_path, "a+") as file:
    for submission_id in submission_id_list[0: 300]:
      file.write(str(submission_id) + ": " + str(submissionid_commentid_map[submission_id]))
      file.write("\n")

In [11]:
# Method-1 - Slower 
def extractCommentIds1(submission_id):
  """This function extracts the list of reply IDs for each submission ID"""
  
  submission_comment_stats[str(submission_id)] = {}

  sub = reddit.submission(id=str(submission_id))

  sub.comments.replace_more(limit=None)
  comment_queue = sub.comments[:]  # Seed with top-level

  comment_ids = []
  while comment_queue:
    comment = comment_queue.pop(0)
    if comment.id:   # To check if the comment is actually extracted 
      comment_conversation_ids[comment.id] = submission_id
      comment_ids.append(comment.id)
    comment_queue.extend(comment.replies)
  submissionid_commentid_map[submission_id] = comment_ids

# Method-2 - Faster and Efficient 
def extractCommentIds2(submission_id="7n2z26"):
  """This function extracts the list of reply IDs for each submission ID"""

  commentids_endpoint = "https://api.pushshift.io/reddit/submission/comment_ids/" + str(submission_id)
  print(commentids_endpoint)
  r = requests.get(commentids_endpoint)
  data = json.loads(r.text)
  submissionid_commentid_map[submission_id] = data["data"]

In [ ]:
# Fix the issue of "None" parent ID
# Fix the issue of those reply_to IDs that are not in the index. Remove the entire tree following it. All instances for which "reply_to" is a particular ID

In [12]:
def extractCommentData(comment_id):   # Pass a save path to the function 

  """This function extracts metadata for each comment_ID"""
  
  try:
    comment_metadata = get_PSAW_Comment_Data(comment_id)
  except:
    time.sleep(5)
    comment_metadata = get_PSAW_Comment_Data(comment_id)

  comment_data = []

  if comment_metadata != []:
    comment = comment_metadata[0]
    id = comment["id"]
    author = comment["author"]
    body = comment["body"]
    created_at = comment["created_utc"]
    permalink = comment["permalink"]

    try:
      parent_id = comment["parent_id"]

    except:
      parent_id = "None"

    score = comment["score"]
    
  # try:
  #   link_id = comment_metadata["link_id"]
  # except:
  #   link_id = "None"

   
  # subreddit = comment_metadata["subreddit"]
  # subreddit_id = comment_metadata["subreddit_id"]

    comment_data.append((author, body, created_at, permalink, parent_id, score))


    submission_comment_stats[str(id)] = comment_data

    # Open and write the 

In [13]:
def retrieveCommentIdData(filepath):  
  """ This function reads list of comment IDs for each submission in the corpus """

  # Save the current status of comment data to file if required 
  # Change the code in exception block after the directory has been read
  # try:
  #   INPUT_DIR = "Something"
  #   filepath = INPUT_DIR
  # except:
  #   print("Path not found")

  with open(str(filepath), "r+") as submission_comment_map_file:
    submission_comment_map_data = submission_comment_map_file.readlines()

    for text in tqdm(submission_comment_map_data):
      text = str(text)
      submission_id_key = re.search("\w+\:", text)
      submission_id = text[submission_id_key.span()[0]: submission_id_key.span()[1]-1]
      comment_id_text = text[9:-2]
      comment_list  = comment_id_text.split(", ")

      # Open the file here. Rectify this tomorrow 
      for comment_id in comment_list:
        comment_id = re.sub("'", "", comment_id)
        comment_conversation_ids[comment_id] = submission_id
        all_comment_ids.append(comment_id)
        extractCommentData(comment_id)    # Return instead of a global call - I mean I can have this code across two files. I can automate the stuff - just see 


In [14]:
def readSubmissionIds(filepath):
  """ This function stores the submission and its metadata 
  The data needs to be appended to existing dataframe of comments so as to enable conversation tree build within Convokit framework 
  """
  data = pd.read_csv(str(filepath))
  data.columns = data.columns.str.replace(" ", "")
  data = data[data["num_comms"] > 0]
  
  # data.drop(["Unnamed: 0", "index"], axis =1, inplace = True) # This step depends on the structure of your CSV. If I structure the CSV as having those column names as defined in submission dataframe, then creating the submission dataframe isn't exactly required. Step-2 can be skipped 
  # data = data.drop_duplicates()
  
  for idx in tqdm(data.index):
    submission_id = data.loc[idx, "post_id"]
    submission_id_list.append(submission_id)
    speaker = data.loc[idx, "speaker"]
    score = data.loc[idx, "score"] 

    timestamp = data.loc[idx, "timestamp"]
    # timestamp = int(time.mktime(timestamp.timetuple()))
    timestamp = int(time.mktime(datetime.strptime(timestamp, '%d-%m-%Y %H:%M').timetuple()))
    num_comms = data.loc[idx, "num_comms"]
    text = data.loc[idx, "text"]

    submission_metadata[str(submission_id)] = [(speaker, timestamp, score, num_comms, text)]

In [15]:
# It runs the parent ID for each comment ID 
def parentId(id):
  return str(reddit.comment(str(id)).parent_id)

## STEP-1
* This action would fill submission_id_list and submission_metadata

In [20]:
# Move all global variables to main function later on x`
submissionid_commentid_map = {}    # Creates submission comment mapping for each submission ID in the corpus
submission_comment_stats = {}      # Stores comment metadata for each comment ID in the corpus 
comment_conversation_ids = {}      # Stores conversation IDs for each comment in the corpus 
all_comment_ids = []               # Stores the list of all comment IDs in a given corpus 
submission_id_list = []            # Stores submission IDs for each corpus 
submission_metadata = {}           # Stores submission metadata for each submission in the corpus

In [21]:
data = pd.read_csv("/content/drive/MyDrive/CP/CP-Sem-8/Datasets/Reddit/Data/Reddit/HS and MA /PussyPass/subs_Pussypass_2021.csv")
data.rename(columns = {"timestamp ": "timestamp"}, inplace = True)

In [22]:
# Change the code in exception block after the directory has been read
INPUT_DIR = "/content/drive/MyDrive/CP/CP-Sem-8/Datasets/Reddit/Data/Reddit/HS and MA /PussyPass/"
filepath = "/content/drive/MyDrive/CP/CP-Sem-8/Datasets/Reddit/Data/Reddit/HS and MA /PussyPass/subs_Pussypass_2021.csv"
readSubmissionIds(filepath)  

100%|██████████| 645/645 [00:00<00:00, 9226.27it/s]


## STEP-2
* This action would store all submission IDs and their corresponding metadata in a dataframe. It is formatted to append it with utterances dataframe while converting into Convokit format

In [23]:
submission_data_df = pd.DataFrame(index = submission_id_list[0: 300], columns = ["speaker", "text", "reply_to", "timestamp", "score", "conversation_id"])
for submission_id in tqdm(submission_id_list[0: 300]):
  submission_data_df.loc[submission_id, "speaker"] = submission_metadata[submission_id][0][0]
  submission_data_df.loc[submission_id, "text"] = submission_metadata[submission_id][0][4]
  submission_data_df.loc[submission_id, "reply_to"] = "None"
  submission_data_df.loc[submission_id, "timestamp"] = submission_metadata[submission_id][0][1]
  submission_data_df.loc[submission_id, "score"] = submission_metadata[submission_id][0][2]
  submission_data_df.loc[submission_id, "conversation_id"] = submission_id

submission_data_df = submission_data_df.drop_duplicates()

100%|██████████| 300/300 [00:00<00:00, 2336.80it/s]


## STEP-3:
*  This action stores all comment IDs for a particular submission ID. It would fill submissionid_commentid_map and comment_conversation_ids

In [ ]:
for submission_id in tqdm(submission_id_list[0: 300]):
  try:
    extractCommentIds2(submission_id)
  except:
    time.sleep(5)
    extractCommentIds2(submission_id)

  0%|          | 0/300 [00:00<?, ?it/s]

https://api.pushshift.io/reddit/submission/comment_ids/koa96t


  0%|          | 1/300 [00:04<22:59,  4.61s/it]

https://api.pushshift.io/reddit/submission/comment_ids/kobq8d


  1%|          | 2/300 [00:08<21:55,  4.41s/it]

...

https://api.pushshift.io/reddit/submission/comment_ids/latv5r


 32%|███▏      | 97/300 [08:40<16:06,  4.76s/it]

https://api.pushshift.io/reddit/submission/comment_ids/latvi2


 33%|███▎      | 98/300 [08:46<17:52,  5.31s/it]

https://api.pushshift.io/reddit/submission/comment_ids/lavm15


## STEP-4:
* This action would store all submission IDs along with their comment IDs to a file 

In [25]:
try:
  root_path = "/content/drive/MyDrive/CP/CP-Sem-8/Datasets/ProcessedData/ProcessedSubsCommentIds/Hate-Speech /PussyPass/"   
  root_path_1 = "/content/sample_data/"
  saveCommentIds(root_path_1)
except Exception as e:
  print(e)

Enter file name to save the submission Id and comment Ids
Don't forget to add the .txt extension. I repeat, add the .txt extension


KeyboardInterrupt: ignored

## STEP-5: 
* Read file contaning submission and commenst IDs
* This action would fill submission_comment_stats 

In [ ]:
path = "/content/drive/MyDrive/CP/CP-Sem-8/Datasets/ProcessedData/ProcessedSubsCommentIds/Hate-Speech /PussyPass/subs_comms_2021.txt"   # Change directory where you have saved your submissions alongwith comments IDs
path_1 = "/content/sample_data/subs_comms_cond_1.txt"
read_filepath = path_1
retrieveCommentIdData(read_filepath)        # If not all comments are to be extracted at a go, create the dataframe below 

In [ ]:
# Save the current status of comment data to file if required 
# Change the code in exception block after the directory has been read
try:
  INPUT_DIR = "Something"
  filepath = INPUT_DIR
except:
  print("Path not found")

In [ ]:
with open(filepath, "a+") as file:
  for comment_id in list(submission_comment_stats.keys()):
    file.write(str(comment_id) + ": " + str(submission_comment_stats[str(comment_id)]))
    file.write("\n")

## STEP-6
* Save the comment IDs and their metadata to disk. Mostly it won't be required as the dataframe would be created instantly and that can be saved as .csv

* Do note the status in each run 


In [ ]:
comment_key_list = list(submission_comment_stats.keys())
comment_data_df = pd.DataFrame(index = comment_key_list, columns = ["speaker", "text", "reply_to", "timestamp", "score"])
for index in tqdm(comment_data_df.index):
  if index in submission_comment_stats.keys():                  # Keep a track of indices so not to restart the process again 
    comment_data_df.loc[index, "speaker"] = submission_comment_stats[str(index)][0][0]
    comment_data_df.loc[index, "text"] = submission_comment_stats[str(index)][0][1]

    # try:
    #   parent_sub_id = 
    
    #   if str(index) != str(parent_sub_id):
    #     df.loc[str(index), "reply_to"] = parent_sub_id   # Check if a better way exists. Search for it in PSAW Docs 
        
    # except:
    #   df.loc[str(index), "reply_to"] = "None"

    comment_data_df.loc[index, "timestamp"] = submission_comment_stats[str(index)][0][2]

    # perma_text = submission_comment_stats[str(index)][0][4]
    # sub_text = perma_text[1:-2].split("/")[3]
    comment_data_df.loc[index, "reply_to"] = submission_comment_stats[str(index)][0][4]

    comment_data_df.loc[index, "score"] = submission_comment_stats[str(index)][0][5]
    # df.loc[index, "subreddit"] = "None"
    # df.loc[index, "subreddit_id"] = "None"

    try:
      comment_data_df.loc[index, "conversation_id"] = comment_conversation_ids[str(index)]
    except:
      comment_data_df.loc[index, "conversation_id"] = "None"

100%|██████████| 88/88 [00:00<00:00, 784.47it/s]


In [ ]:
# submission_data_df.to_csv("/content/sample_data/submission_data_df_2021.csv")

In [ ]:
# comment_data_df.to_csv("/content/sample_data/comment_df_2021.csv")

## STEP-7:
**Some pre-processing for dataframe of utterances**

There are three primary challenges encountered while extracting comment metadata:
1. Parent ID is retrieved in base-10 as well as base-36 format for different API calls
2. Parent ID parameter is missing in some API calls
3. Data list is null - {PSAW Internal problem} while retrieving data OR comment has been deleted


Solutions:
1. We need to have parent IDs in base-36 as per the original Reddit format. Hence we would convert all base-10 ones to base-36 ones 
2. If parent ID parameter is missing, retrieve its parent using PRAW function 
3. Sometimes, we might not get the required data AND can't store its post. 
Some comment IDs would have a parent ID which is itself missing in the post ID.
In such a scenario, for the time being, we would exclude such conversations. Future scenario has been coded in STEP-8

In [ ]:
# Remove all duplicates from comment_ids_df 
comment_data_df = comment_data_df.drop_duplicates()

In [ ]:
comment_data_df["reply_to_new"] = comment_data_df["reply_to"]  # Create a new column that stores parent ID for each post ID 

In [ ]:
comment_data_df.to_csv("/content/sample_data/cond_1.csv")

In [ ]:
for idx in comment_data_df.index:      # Convert parentIDs that are not "None" in base-10 to base-36
  if comment_data_df.loc[idx, "reply_to"] != "None":
    comment_data_df.loc[idx, "reply_to_new"] = str(base36.dumps(int(comment_data_df.loc[idx, "reply_to"]))).lower()

In [ ]:
parent_ids = []

def parentId(id):       # Returns parent IDs for comments for which the "reply_to" parameter is "None"
  if id.startswith("t1_", 0, 2):
    id = id[3: ]
  return (reddit.comment(id).parent_id)[3: ]

In [ ]:
comment_data_df = pd.read_csv("/content/sample_data/askreddit_data.csv", index_col = "Unnamed: 0")

In [ ]:
comment_data_df = comment_data_df.set_index("Unnamed: 0.1")

In [ ]:
comment_data_df.index

Index(['e18uydw', 'e18wieo', 'e1xj23o', 'e1xja8s', 'e1xkf4e', 'e6a950v',
       'dzt5ka3', 'dzt7jsm', 'dztdp4v', 'e3vvse1',
       ...
       'e50nxxn', 'e50o0pp', 'e50otnh', 'e50p6hi', 'e50pvng', 'e69hfmr',
       'e1bac43', 'e1bag2j', 'e1banbr', 'e1baqip'],
      dtype='object', length=1452)

In [ ]:
for com_id in tqdm(comment_data_df.index):                       # Perform conversion 
  if comment_data_df.loc[com_id, "reply_to_new"] == "None":      # The ClientException is raised when the comment has been deleted. No parent id for the deleted comment
    try:
      comment_data_df.loc[com_id, "reply_to_new"] = parentId(com_id)
    except:
      time.sleep(5)
      comment_data_df.loc[com_id, "reply_to_new"] = parentId(com_id)

100%|██████████| 6416/6416 [1:01:57<00:00,  1.73it/s]


In [ ]:
comment_data_df.to_csv("/content/sample_data/comms_pussypass_2018.csv")

In [ ]:
# Rename "reply_to_new" to "reply_to" AND "reply_to" a "reply_to_old". Retain the older column for future reference if required 
cols_rename = {"reply_to": "reply_to_old", "reply_to_new": "reply_to"}
comment_data_df.rename(columns = cols_rename, inplace = True)

In [ ]:
# Drop old "reply_to" values 
comment_data_df.drop(["reply_to_old"], axis = 1, inplace = True)

In [ ]:
# Sort columns in submission and comment dataframes 
comment_data_df_copy = comment_data_df.reindex(sorted(comment_data_df.columns), axis=1)
submission_data_df_copy = submission_data_df.reindex(sorted(submission_data_df.columns), axis=1)
# comment_data_df_copy = comment_data_df_copy.reindex(sorted(comment_data_df.columns), axis=1)
# submission_data_df_copy = subm_data.reindex(sorted(subm_data.columns), axis=1)

In [ ]:
given_num_comms = list(submission_data_df["num_comms"])
submission_data_df.drop("num_comms", axis = 1, inplace = True)

In [ ]:
# Check certain stats 
comment_data_df_copy.isnull().sum()

conversation_id    0
reply_to           0
score              0
speaker            0
text               0
timestamp          0
dtype: int64

In [ ]:
# Check certain stats
submission_data_df_copy.isnull().sum()

conversation_id    0
reply_to           0
score              0
speaker            0
text               0
timestamp          0
dtype: int64

In [ ]:
# Check that each conversation_id in comment dataframe is present in submission_id of parent dataframe 
for conv_id in comment_data_df_copy["conversation_id"].unique():
  if conv_id not in submission_data_df_copy["id"].unique():
    print(conv_id)

In [ ]:
subm_data = pd.read_csv("/content/subs_mensrights_sample.csv", infer_datetime_format = True)
subm_data.drop(["text"], axis = 1, inplace = True)

In [ ]:
# subm_data["timestamp"] = pd.to_datetime(subm_data["timestamp"])

In [ ]:
# for idx in subm_data.index:
#   subm_data.loc[idx, "timestamp"] = int(time.mktime(datetime.strptime(subm_data.loc[idx, "timestamp"], '%Y-%m-%d %H:%M:%S').timetuple()))
# time.mktime(datetime.strptime(ti, '%m/%d/%Y %I:%M %p').timetuple())

In [ ]:
# comment_data_df_copy.to_csv("/content/drive/MyDrive/CP Sem-8-20220801T153939Z-001.zip (Unzipped Files)/CP Sem-8/Datasets/ProcessedData/SubComsText/comment_df_1.csv")

## STEP-8
* Merge the submission DataFrames and commennt dataframes into a new dataframe 
* Create Convokit Corpus 

You can generate corpora from utterance data only. 

(Note that 'id' does not have to be the DataFrame index, it can just be another column in your dataframe.)


### DISCLAIMER
* There should be no "None" values except for those is "reply_to" columns that have sumbmission IDs as index. Drop rest of the rows


In [ ]:
comment_data_df_copy = comment_data_df_copy.reset_index()
comment_data_df_copy.rename(columns = {"index": "id"}, inplace = True)

In [ ]:
submission_data_df_copy.rename(columns = {"post_id": "id"}, inplace = True)

In [ ]:
df_combined = pd.concat([comment_data_df_copy, submission_data_df_copy], axis = 0)

In [ ]:
df_combined = df_combined.reset_index()
df_combined.rename(columns = {"index": "id"}, inplace = True)

In [ ]:
corpus = Corpus(filename = "/content/drive/MyDrive/CP/CP-Sem-8/Datasets/ProcessedData/saved_corpora/mens-rights-2018")
corpus.get_utterances_dataframe()

,timestamp,text,speaker,reply_to,conversation_id,vectors
id,,,,,,
dsdpaj9,1515441265,Nothing wrong with sex.,ee4m,7p12k7,7p12k7,[]
dsdpgka,1515441436,Amount of sexual partners for women before mar...,FyrW,dsdpaj9,7p12k7,[]
dsdpqf5,1515441710,"Sure, thats a non brainer. People that like se...",ee4m,dsdpgka,7p12k7,[]
dsdprlw,1515441742,A lock that opens for any key is useless.\n\nA...,ItSeemedObvious,7p12k7,7p12k7,[]
dsdq5c6,1515442132,&gt; People that like sex like sex so are more...,FyrW,dsdpqf5,7p12k7,[]
...,...,...,...,...,...,...
8b5wj6,1523346368,Judge allows NCFM to challenge male-only draft,PeterWrightMGTOW,None,8b5wj6,[]
8s8ybg,1529415750,When is going to be okay to stop focusing one ...,Ok_Interest,None,8s8ybg,[]
8azojl,1523290689,For those who do not think it happens to men. ...,furchfur,None,8azojl,[]


In [ ]:
# Save the entire data as csv OR create the Convokit corpus 
corpus = Corpus.from_pandas(df_combined) # Else create separate dataframes for conversation, speaker and utterances 

8354it [00:00, 11230.75it/s]


In [ ]:
for conv_id in tqdm(df_combined["conversation_id"].unique()):      # Add these details to metadata
  convo = corpus.get_conversation(conv_id)
  convo.add_meta("original_no_of_comms", submission_metadata[str(conv_id)][0][3]) 
  convo.add_meta("retrieved_no_of_comms", len(convo.get_utterance_ids()))
  convo.add_meta("to_include", convo.check_integrity())
  # print(convo.check_integrity())

In [ ]:
# for id in list(df_combined["reply_to"]):
#   if id != "None":
#     if id not in list(df_combined["id"]):
#       print("Yes")
#       break

In [ ]:
# remove_post_ids = []
# for id in list(comment_data_df["reply_to"]):
#   if id not in list(df_combined["post_id"]) or id == "[deleted]":
#     remove_ids.append(id)

In [ ]:
corpus.random_conversation().print_conversation_structure()

PeterWrightMGTOW
    tenchineuro
        Source_or_gtfo
            tenchineuro
                kaliwraith
                    tenchineuro
            tenchineuro
                Source_or_gtfo
                    tenchineuro
        Vandechoz
            tenchineuro
    WoodChucking
        Unused_UserID
    IronJohnMRA
        IronJohnMRA
    dukunt
    XxRandomguyxX
        pintocat
            Bing_Bang_Bam
    GreyTortoise
        tenchineuro
            GreyTortoise


## STEP-8-Optional - though it would be good to check if integrity constraint works 
* Includes integrity checks for certain corpora and code for removing tree branches from a conversation 

In [ ]:
# Check integrity if the conversation tree structure has been created 
# reply_to column should not be "None" for commentIDs. Remember the tree structure. Convokit first identifies the conversation ID based on the submission ID - followed by its presence in reply_to column in a recursive fashion

try:
  conversation_id = "9mreyq"
  convo = corpus.get_conversation(conversation_id)
  a = convo.check_integrity()
  print(a)

except:
  print("Not defined")

Checking reply-to chain of Conversation 9mreyq
No issues found.

True


In [ ]:
convo = corpus.get_conversation('a9c2aq')
print(convo.print_conversation_structure())

epic_pants44
    downundergoldbon
        epic_pants44
    throwaway_ftb
        epic_pants44
            sphinx2626
    Bluebird_North
        epic_pants44
        PapaLoMein
    mikesteane
    aesopstortoise
        epic_pants44
    Hadashi_blacksky
        epic_pants44
            Hadashi_blacksky
    HerbHandsBill
        epic_pants44
            HerbHandsBill
                epic_pants44
                    HerbHandsBill
    Nicklovn
        epic_pants44
    20cansofSpam
        epic_pants44
    Men-Are-Human
        epic_pants44
            Men-Are-Human
    HerbHandsBill
        epic_pants44
None


## STEP-9

In [ ]:
# Dump corpus and load 
# Run all relevant commands TO PERFORM if the conversion is successful
try:
  CORPUS_NAME = "mens-rights-2019" 
  BASE_PATH = "/content/drive/MyDrive/CP/CP-Sem-8/Datasets/ProcessedData/saved_corpora/"
  corpus.dump(CORPUS_NAME, base_path=BASE_PATH)
except:
  print("Corpus not found")

Corpus not found


In [ ]:
# Load the Corpus
try:
  CORPUS_PATH = "/content/drive/MyDrive/CP/CP-Sem-8/Datasets/ProcessedData/saved_corpora/mens-rights-2019"
  corpus = Corpus(filename = CORPUS_PATH)
except:
  print("Corpus not found")

Corpus not found


In [ ]:
corpus.random_conversation().print_conversation_structure()

Nvr2Much
    thrway_1000
        antilopes
            TracyMorganFreeman
                revengeofthedirty47
                antilopes
    89peters
        TracyMorganFreeman
            89peters
    Vman2
        iburiedmyshovel
    runninginacyclone
        Meistermalkav
        timdesuyo
    runninginacyclone
        [deleted]
    svenskbitch
    ThePigmanAgain
    Mode1961
    [deleted]
    5th_Law_of_Robotics
    IronJohnMRA
        antilopes
    youcantduckthezucc
    CptFizz
    cankle_h8r
    mybuttiswaytoosmall


In [ ]:
submission_data_df_copy.to_csv("/content/sample_data/submission_data_mensrights_2019.csv")

1. The below code can take care of conversations for which integrity check fails
2. Again, it would be efficient only for those conversations where only a minor percentage of all comments for a conversation have their parent IDs as missing. If it is significant, its better to extract more conversations

In [ ]:
from collections import defaultdict
graph = {}
visited = set()

In [ ]:
child_list = list(df_combined["id"]) 
parent_list = list(df_combined["reply_to"])

In [ ]:
for val in parent_list:
  if val == "None":
    print("Yes")

In [ ]:
len(parent_list)

5790

In [ ]:
# Create the Adjaceny list: format is "{"parent_id1": [children], "parent_id2": [children], ...}"
graph_list = []
for child, parent in zip(child_list, parent_list):
  # id = df_combined.loc[idx, "id"]
  # parent_id = df_combined.loc[idx, "reply_to"].values[0]
  # graph_list.append((id, parent_id))
  
  # print(child)
  # print(parent)
  if parent != "None":
    if parent not in graph.keys():
      graph[str(parent)] = []
      graph[str(parent)].append(str(child))
    else:
      graph[str(parent)].append(str(child))
# for child, parent in graph_list:
#   print(child)
#   print(parent)
#   break

In [ ]:
# DFS for storing the entire depth of the sub-tree following the particular deleted utterance
def DFSutil(id, visited):
  if id not in visited:
    visited.add(id)

  if id in list(graph.keys()):
    for idx in graph[id]:
      DFSutil(id, visited)
  else:
    return 

# Now wherever you find that the "parent_id" for a particular id is "[deleted]". Remove that comment and the entire tree following it

for idx, row in df_combined.iterrows():
 
  if row["speaker"] == "[deleted]": 
       id = row["id"]
       DFSutil(id, visited)